In [6]:
from probabilistic_classifier.dataset import create_probabilistic_classifier_dataset_gaussian, create_test_dataset_gaussian, create_batched_tensors
from probabilistic_classifier.probabilistic_classifier import ProbabilisticClassifier
import torch
import torch.nn as nn
from torch.optim import Adam
import torch.nn.functional as F  

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.01
num_of_iterations = 100
batch_size = 2048
num_class = 2
num_features = 2

model = ProbabilisticClassifier(num_features, [100, 100], num_class).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=lr, weight_decay=0.001)

In [3]:
data, label = create_probabilistic_classifier_dataset_gaussian([0, 0], [[1, 0.8], [0.8, 1]], 1000000)
data, label = create_batched_tensors(data, label, batch_size)

In [9]:
running_loss = []
running_avg_loss = 0
num_joint = 0
num_marginal = 0
model.train()
for iter_idx in range(num_of_iterations): 
    for inner_iter_idx, (batched_data, batched_label) in enumerate(zip(data, label)):
        num_marginal = num_marginal + torch.count_nonzero(batched_label)
        num_joint = num_joint + (len(batched_label) - torch.count_nonzero(batched_label))
        batched_label = F.one_hot(batched_label)
        
        batched_data, batched_label = batched_data.to(device), batched_label.to(device)

        optimizer.zero_grad()

        estimated_probabilities = model(batched_data)
        loss = criterion(estimated_probabilities, batched_label.float())
        loss.backward()
        optimizer.step()

        running_loss.append(loss.item())
        running_avg_loss = running_avg_loss + loss.item()

        if inner_iter_idx == 0 or (inner_iter_idx + 1) % 50 == 0:
            print('iter: {}, inner iter: {}, avg. loss: {}'.format(iter_idx + 1, inner_iter_idx + 1, running_avg_loss / len(running_loss)))          

iter: 1, inner iter: 1, avg. loss: 0.6981425285339355
iter: 1, inner iter: 50, avg. loss: 0.5748464620113373
iter: 1, inner iter: 100, avg. loss: 0.5682049840688705
iter: 1, inner iter: 150, avg. loss: 0.5660561382770538
iter: 1, inner iter: 200, avg. loss: 0.5651517909765243
iter: 1, inner iter: 250, avg. loss: 0.5646469376087189
iter: 1, inner iter: 300, avg. loss: 0.5647930421431859
iter: 1, inner iter: 350, avg. loss: 0.5646251521791731
iter: 1, inner iter: 400, avg. loss: 0.5643049027025699
iter: 1, inner iter: 450, avg. loss: 0.5642203549544017
iter: 2, inner iter: 1, avg. loss: 0.5641397803413625
iter: 2, inner iter: 50, avg. loss: 0.5641599060874259
iter: 2, inner iter: 100, avg. loss: 0.5638746818581339
iter: 2, inner iter: 150, avg. loss: 0.5636836192417592
iter: 2, inner iter: 200, avg. loss: 0.5635589253054648
iter: 2, inner iter: 250, avg. loss: 0.563462872547129
iter: 2, inner iter: 300, avg. loss: 0.5635499299252441
iter: 2, inner iter: 350, avg. loss: 0.5635646053093694

In [10]:
p_joint = (num_joint / (num_joint + num_marginal)).item()
p_marginal = (num_marginal / (num_joint + num_marginal)).item()
print(p_joint, p_marginal)

0.5 0.5


In [18]:
with torch.no_grad():
    model.eval()
    mi = 0
    for i in range(1000):
        test_data = create_test_dataset_gaussian([0, 0], [[1, 0.8], [0.8, 1]], 256)
        test_data = torch.from_numpy(test_data).to(torch.float32)
        test_data = test_data.to(device)
        estimated_test_probabilities = model(test_data)
        
        estimated_test_probabilities = torch.sigmoid(estimated_test_probabilities)
        joint, marginal = estimated_test_probabilities[:, 0], estimated_test_probabilities[:, 1]
        mi += torch.sum(torch.log(torch.div(joint, marginal))) / 256

In [19]:
mi / 1000

tensor(0.5210, device='cuda:0')